<a href="https://colab.research.google.com/github/ngaivoronski/Algorithms/blob/master/unixFileSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from abc import ABC, abstractmethod
from collections import deque

class File():
    def __init__(self, name, size):
        self.name = name
        self.size = size
        self.children = []
        self.isDirectory = '.' not in name
        self.extension = name.split('.')[-1] if not self.isDirectory else ""

    def __repr__(self) -> str:
        return self.name

class Filter(ABC):
    @abstractmethod
    def apply(self, file):
        pass

class SizeFilter(Filter):
    def __init__(self, size):
        self.size = size

    def apply(self, file):
        return file.size >= self.size

class ExtensionFilter(Filter):
    def __init__(self, extension):
        self.extension = extension

    def apply(self, file):
        return file.extension == self.extension

class AndFilter(Filter):
    def __init__(self, *filters):
        self.filters = filters

    def apply(self, file):
        return all(f.apply(file) for f in self.filters)

class OrFilter(Filter):
    def __init__(self, *filters):
        self.filters = filters

    def apply(self, file):
        return any(f.apply(file) for f in self.filters)

class ApplySearch():
    def __init__(self, filter=None):
        self.filter = filter

    def setFilter(self, filter):
        self.filter = filter

    def find(self, root):
        result = []
        queue = deque([root])
        while queue:
            current = queue.popleft()
            if current.isDirectory:
                queue.extend(current.children)
            else:
                if self.filter and self.filter.apply(current):
                    result.append(current)
        return result

f1 = File("root_300", 300)

f2 = File("fiction_100", 100)
f3 = File("action_100", 100)
f4 = File("comedy_100", 100)
f1.children = [f2, f3, f4]

f5 = File("StarTrek_4.txt", 4)
f6 = File("StarWars_10.xml", 10)
f7 = File("JusticeLeague_15.txt", 15)
f8 = File("Spock_1.jpg", 1)
f2.children = [f5, f6, f7, f8]

f9 = File("IronMan_9.txt", 9)
f10 = File("MissionImpossible_10.rar", 10)
f11 = File("TheLordOfRings_3.zip", 3)
f3.children = [f9, f10, f11]

f11 = File("BigBangTheory_4.txt", 4)
f12 = File("AmericanPie_6.mp3", 6)
f4.children = [f11, f12]


greater5_filter = SizeFilter(5)
txt_filter = ExtensionFilter("txt")
rar_filter = ExtensionFilter("rar")

searcher = ApplySearch(OrFilter(greater5_filter, txt_filter))
print(searcher.find(f1))

searcher = ApplySearch(AndFilter(greater5_filter, txt_filter))
print(searcher.find(f1))

searcher = ApplySearch(AndFilter(OrFilter(rar_filter, txt_filter),greater5_filter))
print(searcher.find(f1))


[StarTrek_4.txt, StarWars_10.xml, JusticeLeague_15.txt, IronMan_9.txt, MissionImpossible_10.rar, BigBangTheory_4.txt, AmericanPie_6.mp3]
[JusticeLeague_15.txt, IronMan_9.txt]
[JusticeLeague_15.txt, IronMan_9.txt, MissionImpossible_10.rar]
